In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns',None)
%config InlineBackend.figure_format = 'svg'

In [2]:
df_1 = pd.read_csv('data/sofa_hourly_cluster.csv')

In [3]:
df_1

,stay_id,cluster
0,30000646,4
1,30001471,0
2,30003087,0
3,30003598,2
4,30004018,1
...,...,...
22435,39998012,1
22436,39998622,4
22437,39999230,0
22438,39999562,4


In [4]:
df_2 = pd.read_parquet('data/icustays.parquet')

In [5]:
df_2.head()

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,10000032,29079034,39553978,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266
1,10000980,26913865,39765666,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535
2,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032
3,10001217,27703517,34592300,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-12-19 15:42:24,2157-12-20 14:27:41,0.948113
4,10001725,25563031,31205490,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588


In [6]:
# los shall be dropped due to info leakage
df = pd.merge(df_1, df_2.drop(columns = ['los', 'intime', 'outtime']), on = 'stay_id')

In [7]:
df

,stay_id,cluster,subject_id,hadm_id,first_careunit,last_careunit
0,30000646,4,12207593,22795209,Coronary Care Unit (CCU),Coronary Care Unit (CCU)
1,30001471,0,17461994,29537385,Neuro Intermediate,Neuro Intermediate
2,30003087,0,10682002,20035892,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU)
3,30003598,2,15332791,20683754,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU)
4,30004018,1,11101925,23920532,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU)
...,...,...,...,...,...,...
22435,39998012,1,17874908,25979104,Trauma SICU (TSICU),Trauma SICU (TSICU)
22436,39998622,4,19046950,24352151,Trauma SICU (TSICU),Trauma SICU (TSICU)
22437,39999230,0,13651601,22584645,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU)
22438,39999562,4,15403458,25335698,Neuro Intermediate,Neuro Intermediate


In [8]:
df.first_careunit.unique()

array(['Coronary Care Unit (CCU)', 'Neuro Intermediate',
       'Medical Intensive Care Unit (MICU)',
       'Surgical Intensive Care Unit (SICU)', 'Trauma SICU (TSICU)',
       'Cardiac Vascular Intensive Care Unit (CVICU)',
       'Medical/Surgical Intensive Care Unit (MICU/SICU)',
       'Neuro Surgical Intensive Care Unit (Neuro SICU)',
       'Neuro Stepdown'], dtype=object)

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df.first_careunit = le.fit_transform(df.first_careunit)
df.last_careunit = le.fit_transform(df.last_careunit)

In [10]:
df

,stay_id,cluster,subject_id,hadm_id,first_careunit,last_careunit
0,30000646,4,12207593,22795209,1,1
1,30001471,0,17461994,29537385,4,4
2,30003087,0,10682002,20035892,2,2
3,30003598,2,15332791,20683754,7,7
4,30004018,1,11101925,23920532,2,2
...,...,...,...,...,...,...
22435,39998012,1,17874908,25979104,8,8
22436,39998622,4,19046950,24352151,8,8
22437,39999230,0,13651601,22584645,2,2
22438,39999562,4,15403458,25335698,4,4


In [11]:
df.subject_id.nunique()

18155

In [12]:
df.hadm_id.nunique()

20934

In [13]:
df.stay_id.nunique()

22440

In [14]:
df_3 = pd.read_parquet('data/admissions.parquet')

In [15]:
df.head()

,stay_id,cluster,subject_id,hadm_id,first_careunit,last_careunit
0,30000646,4,12207593,22795209,1,1
1,30001471,0,17461994,29537385,4,4
2,30003087,0,10682002,20035892,2,2
3,30003598,2,15332791,20683754,7,7
4,30004018,1,11101925,23920532,2,2


In [16]:
df_3.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaT,URGENT,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaT,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaT,EW EMER.,EMERGENCY ROOM,HOSPICE,Medicaid,ENGLISH,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaT,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0
4,10000980,20897796,2193-08-15 01:01:00,2193-08-17 15:07:00,NaT,OBSERVATION ADMIT,WALK-IN/SELF REFERRAL,HOME HEALTH CARE,Other,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,2193-08-14 21:25:00,2193-08-15 02:22:00,0


In [17]:
df_3.columns

Index(['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime',
       'admission_type', 'admission_location', 'discharge_location',
       'insurance', 'language', 'marital_status', 'race', 'edregtime',
       'edouttime', 'hospital_expire_flag'],
      dtype='object')

In [18]:
df = pd.merge(df, df_3.drop(columns = ['admittime', 'dischtime', 'deathtime', 'discharge_location', \
                                      'edregtime', 'edouttime']), on = ['subject_id', 'hadm_id'])

In [19]:
df.marital_status = df.marital_status.fillna('UNKNOWN')
for i in ['admission_type', 'admission_location', 'insurance', 'language', 'marital_status', 'race']:
    df[i] = le.fit_transform(df[i])

In [20]:
df_4 = pd.read_parquet('data/patients.parquet')

In [21]:
df = pd.merge(df, df_4.drop(columns = ['dod']), on = ['subject_id'])

In [22]:
for i in ['anchor_year_group']:
    df[i] = le.fit_transform(df[i])

In [23]:
df

,stay_id,cluster,subject_id,hadm_id,first_careunit,last_careunit,admission_type,admission_location,insurance,language,marital_status,race,hospital_expire_flag,gender,anchor_age,anchor_year,anchor_year_group
0,30000646,4,12207593,22795209,1,1,3,2,0,0,1,3,1,M,43,2193,1
1,30001471,0,17461994,29537385,4,4,4,8,2,0,1,19,0,M,86,2184,2
2,30003087,0,10682002,20035892,2,2,4,6,2,1,3,27,0,M,55,2132,3
3,30003598,2,15332791,20683754,7,7,3,2,1,1,1,28,0,F,64,2189,0
4,30004018,1,11101925,23920532,2,2,3,2,2,1,1,28,1,F,56,2158,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22435,39996867,0,12478391,28760446,8,2,3,8,2,1,2,28,0,F,41,2165,3
22436,39998012,1,17874908,25979104,8,8,3,2,1,1,1,28,1,F,40,2133,0
22437,39999230,0,13651601,22584645,2,2,3,2,2,1,1,28,0,M,49,2147,0
22438,39999562,4,15403458,25335698,4,4,5,6,1,1,1,28,0,M,69,2127,3


In [24]:
sofa_score_1 = pd.read_parquet('data/sofa_score_day1.parquet').drop(columns = ['subject_id', 'hadm_id'])

In [25]:
sofa_score_1

,stay_id,sofa,respiration,coagulation,liver,cardiovascular,cns,renal
0,30000153,4,0.0,0.0,NaN,0.0,3.0,1.0
1,30000213,8,3.0,0.0,0.0,1.0,1.0,3.0
2,30000646,3,2.0,0.0,0.0,1.0,0.0,0.0
3,30001148,6,1.0,0.0,NaN,1.0,4.0,0.0
4,30001336,1,NaN,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...
73136,39999301,2,NaN,1.0,0.0,0.0,0.0,1.0
73137,39999384,3,NaN,0.0,NaN,1.0,0.0,2.0
73138,39999552,8,3.0,0.0,NaN,1.0,4.0,0.0
73139,39999562,0,NaN,NaN,NaN,0.0,0.0,0.0


In [26]:
sofa_1 = pd.read_csv('data/sofa_day1.csv')
sofa_score_1 = pd.read_parquet('data/sofa_score_day1.parquet').drop(columns = ['subject_id', 'hadm_id'])
oasis_1 = pd.read_csv('data/oasis_day1.csv').drop(columns = ['subject_id', 'hadm_id', 'age', 'age_score'])
sofa_2 = pd.read_csv('data/sofa_day2.csv')
sofa_score_2 = pd.read_parquet('data/sofa_score_day2.parquet').drop(columns = ['subject_id', 'hadm_id'])
oasis_2 = pd.read_csv('data/oasis_day2.csv').drop(columns = ['subject_id', 'hadm_id', 'age', 'age_score'])
sofa_3 = pd.read_csv('data/sofa_day3.csv')
sofa_score_3 = pd.read_parquet('data/sofa_score_day3.parquet').drop(columns = ['subject_id', 'hadm_id'])
oasis_3 = pd.read_csv('data/oasis_day3.csv').drop(columns = ['subject_id', 'hadm_id', 'age', 'age_score'])

sofa_1.columns = ['stay_id'] + [i+'_d1' for i in sofa_1.columns[1:]]
sofa_score_1.columns = ['stay_id'] + [i+'_d1' for i in sofa_score_1.columns[1:]]
oasis_1.columns = ['stay_id'] + [i+'_d1' for i in oasis_1.columns[1:]]
sofa_2.columns = ['stay_id'] + [i+'_d2' for i in sofa_2.columns[1:]]
sofa_score_2.columns = ['stay_id'] + [i+'_d2' for i in sofa_score_2.columns[1:]]
oasis_2.columns = ['stay_id'] + [i+'_d2' for i in oasis_2.columns[1:]]
sofa_3.columns = ['stay_id'] + [i+'_d3' for i in sofa_3.columns[1:]]
sofa_score_3.columns = ['stay_id'] + [i+'_d3' for i in sofa_score_3.columns[1:]]
oasis_3.columns = ['stay_id'] + [i+'_d3' for i in oasis_3.columns[1:]]

df = pd.merge(df, sofa_1, on='stay_id')
df = pd.merge(df, sofa_score_1, on='stay_id')
df = pd.merge(df, oasis_1, on='stay_id')
df = pd.merge(df, sofa_2, on='stay_id')
df = pd.merge(df, sofa_score_2, on='stay_id')
df = pd.merge(df, oasis_2, on='stay_id')
df = pd.merge(df, sofa_3, on='stay_id')
df = pd.merge(df, sofa_score_3, on='stay_id')
df = pd.merge(df, oasis_3, on='stay_id')

In [27]:
df_sep = pd.read_parquet('data/sepsis.parquet')

In [28]:
df = pd.merge(df, df_sep[['stay_id', 'sepsis3']], on='stay_id', how = 'left')

In [29]:
df.sepsis3 = df.sepsis3.map({True: 1}).fillna(0).astype(int)

In [30]:
df

,stay_id,cluster,subject_id,hadm_id,first_careunit,last_careunit,admission_type,admission_location,insurance,language,marital_status,race,hospital_expire_flag,gender,anchor_age,anchor_year,anchor_year_group,pao2fio2ratio_novent_avg_d1,pao2fio2ratio_vent_avg_d1,rate_epinephrine_avg_d1,rate_norepinephrine_avg_d1,rate_dopamine_avg_d1,rate_dobutamine_avg_d1,platelet_min_d1,creatinine_max_d1,bilirubin_max_d1,sofa_d1,respiration_d1,coagulation_d1,liver_d1,cardiovascular_d1,cns_d1,renal_d1,oasis_d1,oasis_prob_d1,preiculos_d1,preiculos_score_d1,gcs_d1,gcs_score_d1,heartrate_d1,heart_rate_score_d1,meanbp_d1,mbp_score_d1,resprate_d1,resp_rate_score_d1,temp_d1,temp_score_d1,urineoutput_d1,urineoutput_score_d1,mechvent_d1,mechvent_score_d1,electivesurgery_d1,electivesurgery_score_d1,pao2fio2ratio_novent_avg_d2,pao2fio2ratio_vent_avg_d2,rate_epinephrine_avg_d2,rate_norepinephrine_avg_d2,rate_dopamine_avg_d2,rate_dobutamine_avg_d2,platelet_min_d2,creatinine_max_d2,bilirubin_max_d2,sofa_d2,respiration_d2,coagulation_d2,liver_d2,cardiovascular_d2,cns_d2,renal_d2,oasis_d2,oasis_prob_d2,preiculos_d2,preiculos_score_d2,gcs_d2,gcs_score_d2,heartrate_d2,heart_rate_score_d2,meanbp_d2,mbp_score_d2,resprate_d2,resp_rate_score_d2,temp_d2,temp_score_d2,urineoutput_d2,urineoutput_score_d2,mechvent_d2,mechvent_score_d2,electivesurgery_d2,electivesurgery_score_d2,pao2fio2ratio_novent_avg_d3,pao2fio2ratio_vent_avg_d3,rate_epinephrine_avg_d3,rate_norepinephrine_avg_d3,rate_dopamine_avg_d3,rate_dobutamine_avg_d3,platelet_min_d3,creatinine_max_d3,bilirubin_max_d3,sofa_d3,respiration_d3,coagulation_d3,liver_d3,cardiovascular_d3,cns_d3,renal_d3,oasis_d3,oasis_prob_d3,preiculos_d3,preiculos_score_d3,gcs_d3,gcs_score_d3,heartrate_d3,heart_rate_score_d3,meanbp_d3,mbp_score_d3,resprate_d3,resp_rate_score_d3,temp_d3,temp_score_d3,urineoutput_d3,urineoutput_score_d3,mechvent_d3,mechvent_score_d3,electivesurgery_d3,electivesurgery_score_d3,sepsis3
0,30000646,4,12207593,22795209,1,1,3,2,0,0,1,3,1,M,43,2193,1,71.0,NaN,NaN,NaN,NaN,NaN,266.0,1.0,0.7,3,2.0,0.0,0.0,1.0,0.0,0.0,22,0.033259,1856.366667,1,15.0,0.0,102.0,1.0,42.0,3.0,38.0,6.0,39.11,2.0,4375.0,0.0,0,0,0,6,NaN,NaN,NaN,NaN,NaN,NaN,230.0,NaN,NaN,1,NaN,0.0,NaN,1.0,0.0,0.0,23,0.037612,1856.366667,1,15.0,0.0,113.0,3.0,56.0,2.0,32.0,6.0,37.000,2.0,5820.0,0.0,0,0,0,6,NaN,NaN,NaN,NaN,NaN,NaN,189.0,0.6,NaN,1,NaN,0.0,NaN,1.0,0.0,0.0,23,0.037612,1856.366667,1,15.0,0.0,119.0,3.0,61.0,2.0,31.0,6.0,36.89,2.0,4200.0,0.0,0,0,0,6,1
1,30001471,0,17461994,29537385,4,4,4,8,2,0,1,19,0,M,86,2184,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,0.0,1.0,0.0,31,0.097783,74.000000,3,13.0,4.0,107.0,3.0,98.0,0.0,27.0,1.0,36.39,2.0,850.0,5.0,0,0,0,6,NaN,NaN,NaN,NaN,NaN,NaN,171.0,1.1,1.1,1,NaN,0.0,0.0,0.0,1.0,0.0,29,0.077479,74.000000,3,13.0,4.0,100.0,1.0,85.5,0.0,27.0,1.0,37.280,2.0,750.0,5.0,0,0,0,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,0.0,1.0,0.0,31,0.097783,74.000000,3,13.0,4.0,108.0,3.0,82.5,0.0,29.0,1.0,37.22,2.0,850.0,5.0,0,0,0,6,0
2,30003087,0,10682002,20035892,2,2,4,6,2,1,3,27,0,M,55,2132,3,NaN,NaN,NaN,NaN,NaN,NaN,280.0,0.7,0.9,3,NaN,0.0,0.0,0.0,3.0,0.0,41,0.279468,510.416667,0,7.0,10.0,141.0,6.0,97.5,0.0,35.0,6.0,36.39,2.0,735.0,5.0,0,0,0,6,NaN,NaN,NaN,NaN,NaN,NaN,265.0,0.7,0.6,3,NaN,0.0,0.0,0.0,3.0,0.0,36,0.170145,510.416667,0,7.0,10.0,115.0,3.0,99.5,0.0,33.0,6.0,36.665,0.0,1255.0,5.0,0,0,0,6,NaN,NaN,NaN,NaN,NaN,NaN,256.0,0.7,0.4,3,NaN,0.0,0.0,0.0,3.0,0.0,36,0.170145,510.416667,0,7.0,10.0,105.0,1.0,96.5,0.0,40.0,6.0,37.00,2.0,1060.0,5.0,0,0,0,6,0
3,30003598,2,15332791,20683754,7,7,3,2,1,1,1,28,0,F,64,2189,0,NaN,302.0,NaN,NaN,NaN,NaN,364.0,1.8,NaN,5,2.0,0.0,NaN,1.0,1.0,1.0,46,0.423212,59590.416667,2,14.0,3.0,112.0,3.0,52.0,2.0,31.0,6.0,35.83,4.0,790.0,5.0,1,9,0,6,NaN,238.000000,NaN,NaN,NaN,NaN,208.0,1.7,NaN,3,0.0,0.0,NaN,1.0,1.0,1.0,39,0.231102,59590.416667,2,14.0,3.0,122.0,3.0,54.0,2.0,34.0,6.0,38.170,2.0,3470.0,0.0,1,9,0,6,NaN,NaN,NaN,NaN,NaN,NaN,218.0,1.5,NaN,3,NaN,0.0,NaN,1.0,1.0,1.0,30,0.087098,59590.416667,2,14

In [32]:
df.to_parquet('data/final_full.parquet')

In [36]:
df.drop(columns = ['cluster']).to_parquet('data/final_full_wo_cluster.parquet')

In [37]:
[i for i in df.columns if 'sofa' in i or 'oasis' in i]

['sofa_d1',
 'oasis_d1',
 'oasis_prob_d1',
 'sofa_d2',
 'oasis_d2',
 'oasis_prob_d2',
 'sofa_d3',
 'oasis_d3',
 'oasis_prob_d3']

In [39]:
df.drop(columns = [i for i in df.columns if 'sofa' in i or 'oasis' in i]).to_parquet('data/final_clin.parquet')

In [40]:
df.drop(columns = [i for i in df.columns if 'sofa' in i or 'oasis' in i] + ['cluster']).to_parquet('data/final_clin_wo_cluster.parquet')

In [51]:
df.drop(columns = [i for i in df.columns if 'sofa' in i or 'oasis' in i] + ['cluster'] +
       [i for i in df.columns if '_d2' in i or '_d3' in i]).to_parquet('data/final_clin_wo_cluster_d2d3.parquet')

In [56]:
df.drop(columns = [i for i in df.columns if 'sofa' in i or 'oasis' in i] + ['cluster'] +
       [i for i in df.columns if '_d2' in i or '_d3' in i])\
    .drop(columns = ['first_careunit', 'last_careunit',
       'admission_type', 'admission_location', 'insurance', 'language',
       'marital_status', 'race', 'gender',
       'anchor_age', 'anchor_year', 'anchor_year_group',
        'preiculos_d1', 'preiculos_score_d1', 'gcs_d1', 'gcs_score_d1',
       'heartrate_d1', 'heart_rate_score_d1', 'meanbp_d1', 'mbp_score_d1',
       'resprate_d1', 'resp_rate_score_d1', 'temp_d1', 'temp_score_d1',
       'urineoutput_d1', 'urineoutput_score_d1', 'mechvent_d1',
       'mechvent_score_d1', 'electivesurgery_d1', 'electivesurgery_score_d1',
       'sepsis3']).to_parquet('data/final_clin_sofa_d1.parquet')

In [55]:
df

,stay_id,cluster,subject_id,hadm_id,first_careunit,last_careunit,admission_type,admission_location,insurance,language,marital_status,race,hospital_expire_flag,gender,anchor_age,anchor_year,anchor_year_group,pao2fio2ratio_novent_avg_d1,pao2fio2ratio_vent_avg_d1,rate_epinephrine_avg_d1,rate_norepinephrine_avg_d1,rate_dopamine_avg_d1,rate_dobutamine_avg_d1,platelet_min_d1,creatinine_max_d1,bilirubin_max_d1,sofa_d1,respiration_d1,coagulation_d1,liver_d1,cardiovascular_d1,cns_d1,renal_d1,oasis_d1,oasis_prob_d1,preiculos_d1,preiculos_score_d1,gcs_d1,gcs_score_d1,heartrate_d1,heart_rate_score_d1,meanbp_d1,mbp_score_d1,resprate_d1,resp_rate_score_d1,temp_d1,temp_score_d1,urineoutput_d1,urineoutput_score_d1,mechvent_d1,mechvent_score_d1,electivesurgery_d1,electivesurgery_score_d1,pao2fio2ratio_novent_avg_d2,pao2fio2ratio_vent_avg_d2,rate_epinephrine_avg_d2,rate_norepinephrine_avg_d2,rate_dopamine_avg_d2,rate_dobutamine_avg_d2,platelet_min_d2,creatinine_max_d2,bilirubin_max_d2,sofa_d2,respiration_d2,coagulation_d2,liver_d2,cardiovascular_d2,cns_d2,renal_d2,oasis_d2,oasis_prob_d2,preiculos_d2,preiculos_score_d2,gcs_d2,gcs_score_d2,heartrate_d2,heart_rate_score_d2,meanbp_d2,mbp_score_d2,resprate_d2,resp_rate_score_d2,temp_d2,temp_score_d2,urineoutput_d2,urineoutput_score_d2,mechvent_d2,mechvent_score_d2,electivesurgery_d2,electivesurgery_score_d2,pao2fio2ratio_novent_avg_d3,pao2fio2ratio_vent_avg_d3,rate_epinephrine_avg_d3,rate_norepinephrine_avg_d3,rate_dopamine_avg_d3,rate_dobutamine_avg_d3,platelet_min_d3,creatinine_max_d3,bilirubin_max_d3,sofa_d3,respiration_d3,coagulation_d3,liver_d3,cardiovascular_d3,cns_d3,renal_d3,oasis_d3,oasis_prob_d3,preiculos_d3,preiculos_score_d3,gcs_d3,gcs_score_d3,heartrate_d3,heart_rate_score_d3,meanbp_d3,mbp_score_d3,resprate_d3,resp_rate_score_d3,temp_d3,temp_score_d3,urineoutput_d3,urineoutput_score_d3,mechvent_d3,mechvent_score_d3,electivesurgery_d3,electivesurgery_score_d3,sepsis3
0,30000646,4,12207593,22795209,1,1,3,2,0,0,1,3,1,M,43,2193,1,71.0,NaN,NaN,NaN,NaN,NaN,266.0,1.0,0.7,3,2.0,0.0,0.0,1.0,0.0,0.0,22,0.033259,1856.366667,1,15.0,0.0,102.0,1.0,42.0,3.0,38.0,6.0,39.11,2.0,4375.0,0.0,0,0,0,6,NaN,NaN,NaN,NaN,NaN,NaN,230.0,NaN,NaN,1,NaN,0.0,NaN,1.0,0.0,0.0,23,0.037612,1856.366667,1,15.0,0.0,113.0,3.0,56.0,2.0,32.0,6.0,37.000,2.0,5820.0,0.0,0,0,0,6,NaN,NaN,NaN,NaN,NaN,NaN,189.0,0.6,NaN,1,NaN,0.0,NaN,1.0,0.0,0.0,23,0.037612,1856.366667,1,15.0,0.0,119.0,3.0,61.0,2.0,31.0,6.0,36.89,2.0,4200.0,0.0,0,0,0,6,1
1,30001471,0,17461994,29537385,4,4,4,8,2,0,1,19,0,M,86,2184,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,0.0,1.0,0.0,31,0.097783,74.000000,3,13.0,4.0,107.0,3.0,98.0,0.0,27.0,1.0,36.39,2.0,850.0,5.0,0,0,0,6,NaN,NaN,NaN,NaN,NaN,NaN,171.0,1.1,1.1,1,NaN,0.0,0.0,0.0,1.0,0.0,29,0.077479,74.000000,3,13.0,4.0,100.0,1.0,85.5,0.0,27.0,1.0,37.280,2.0,750.0,5.0,0,0,0,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,0.0,1.0,0.0,31,0.097783,74.000000,3,13.0,4.0,108.0,3.0,82.5,0.0,29.0,1.0,37.22,2.0,850.0,5.0,0,0,0,6,0
2,30003087,0,10682002,20035892,2,2,4,6,2,1,3,27,0,M,55,2132,3,NaN,NaN,NaN,NaN,NaN,NaN,280.0,0.7,0.9,3,NaN,0.0,0.0,0.0,3.0,0.0,41,0.279468,510.416667,0,7.0,10.0,141.0,6.0,97.5,0.0,35.0,6.0,36.39,2.0,735.0,5.0,0,0,0,6,NaN,NaN,NaN,NaN,NaN,NaN,265.0,0.7,0.6,3,NaN,0.0,0.0,0.0,3.0,0.0,36,0.170145,510.416667,0,7.0,10.0,115.0,3.0,99.5,0.0,33.0,6.0,36.665,0.0,1255.0,5.0,0,0,0,6,NaN,NaN,NaN,NaN,NaN,NaN,256.0,0.7,0.4,3,NaN,0.0,0.0,0.0,3.0,0.0,36,0.170145,510.416667,0,7.0,10.0,105.0,1.0,96.5,0.0,40.0,6.0,37.00,2.0,1060.0,5.0,0,0,0,6,0
3,30003598,2,15332791,20683754,7,7,3,2,1,1,1,28,0,F,64,2189,0,NaN,302.0,NaN,NaN,NaN,NaN,364.0,1.8,NaN,5,2.0,0.0,NaN,1.0,1.0,1.0,46,0.423212,59590.416667,2,14.0,3.0,112.0,3.0,52.0,2.0,31.0,6.0,35.83,4.0,790.0,5.0,1,9,0,6,NaN,238.000000,NaN,NaN,NaN,NaN,208.0,1.7,NaN,3,0.0,0.0,NaN,1.0,1.0,1.0,39,0.231102,59590.416667,2,14.0,3.0,122.0,3.0,54.0,2.0,34.0,6.0,38.170,2.0,3470.0,0.0,1,9,0,6,NaN,NaN,NaN,NaN,NaN,NaN,218.0,1.5,NaN,3,NaN,0.0,NaN,1.0,1.0,1.0,30,0.087098,59590.416667,2,14

In [50]:
len([i for i in df.columns if '_d2' in i or '_d3' in i])

72

In [ ]:
df

In [ ]:
df.dtypes

In [45]:
df.electivesurgery_score_d2.unique()

array([6, 0])

cluster 5
insurance 3
language 2
marital_status 5
hospital_expire_flag 2
gender 2
anchor_year_group 4
respiration_d1 5
coagulation_d1 5
liver_d1 5
cardiovascular_d1 5
cns_d1 5
renal_d1 5
preiculos_score_d1 5
gcs_score_d1 4
heart_rate_score_d1 5
mbp_score_d1 4
resp_rate_score_d1 5
temp_score_d1 5
urineoutput_score_d1 5
mechvent_d1 2
mechvent_score_d1 2
electivesurgery_d1 2
electivesurgery_score_d1 2
respiration_d2 5
coagulation_d2 5
liver_d2 5
cardiovascular_d2 5
cns_d2 5
renal_d2 5
preiculos_score_d2 5
gcs_score_d2 4
heart_rate_score_d2 5
mbp_score_d2 4
resp_rate_score_d2 5
temp_score_d2 5
urineoutput_score_d2 5
mechvent_d2 2
mechvent_score_d2 2
electivesurgery_d2 2
electivesurgery_score_d2 2
respiration_d3 5
coagulation_d3 5
liver_d3 5
cardiovascular_d3 5
cns_d3 5
renal_d3 5
preiculos_score_d3 5
gcs_score_d3 4
heart_rate_score_d3 5
mbp_score_d3 4
resp_rate_score_d3 5
temp_score_d3 5
urineoutput_score_d3 5
mechvent_d3 2
mechvent_score_d3 2
electivesurgery_d3 2
electivesurgery_score_d3 